

In this env, the game is made to turn based. the game always starts with player1 then each player take turn on actions.
Also, the heuristic rule is changed to be mix of cautious and reckless, by a random pick.


In [1]:
import random
import numpy as np
from gym.spaces import Discrete,Tuple, Box

from ray.rllib.env.multi_agent_env import MultiAgentEnv

class LOREnv2(MultiAgentEnv):
    """Two-player environment for league of rookie setup1
    This is a turn based version, the two players take action by turn.
    
    The game happens in a 5x5 2D space. Two players are put into two spots. 
    In each turn, the play can take one of the following actions
    - Move 1 step (one of the 4 directions)
    - Attack the opponent
    
    The observation has the followings.
        - 2D position of self
        - health of self
        - 2D position of the opponent
        - health of the opponent
    """

    # all the actions
    MOVEUP = 0
    MOVEDOWN = 1
    MOVELEFT = 2
    MOVERIGHT = 3
    ATTACK = 4
    
    action_string = {
        MOVEUP: "MoveUp",
        MOVEDOWN: "MoveDown",
        MOVELEFT: "MoveLeft",
        MOVERIGHT: "MoveRight",
        ATTACK: "Attack"
    }
    
    # max heath to start with
    max_health = 3
    
    # space is of size n x n 
    # (0, 0) is at the top left corner
    # x represents the vertical direction
    # y represents the horizontal direction
    space_size_n = 3
    
    # miss rate on any one attack
    attack_miss_rate = 0.2
        
    # each attack takes some health
    attak_power = 1
    
    # reward of win a game
    game_award = 100
    
    hit_wall_penalty = -10
    
        
    def generate_init_pos(self):
        player1_init_pos = [random.randrange(LOREnv2.space_size_n), random.randrange(LOREnv2.space_size_n)]
        player2_init_pos = [random.randrange(LOREnv2.space_size_n), random.randrange(LOREnv2.space_size_n)]
        
        while player1_init_pos == player2_init_pos:
          player2_init_pos = [random.randrange(LOREnv2.space_size_n), random.randrange(LOREnv2.space_size_n)]
        
        return player1_init_pos, player2_init_pos

    def __init__(self, config):
        self.action_space = Discrete(len(LOREnv2.action_string))
        
        # the observation is a tuple: [self_pos_x, self_pos_y, self.health, pos_x, pos_y, health]
        # start with a discrete space
        self.observation_space = Tuple(
            [
                # self position in x/y
                Box(low = 0, high = LOREnv2.space_size_n - 1, shape=(2, ), dtype=np.int16),
                # opponent position in x/y
                Box(low = 0, high = LOREnv2.space_size_n - 1, shape=(2, ), dtype=np.int16),
                # self health and opponent health
                Box(low = 0, high = LOREnv2.max_health, shape=(2, ), dtype=np.int16),
                
            ]
        )
        
        self.player1 = "player1"
        self.player2 = "player2"
        
        # set init position
        self.player1_init_pos, self.player2_init_pos = self.generate_init_pos()
        
        self.position = {
                self.player1: self.player1_init_pos,
                self.player2: self.player2_init_pos
        }
        
        self.health = {
            self.player1: LOREnv2.max_health,
            self.player2: LOREnv2.max_health
        }
        
        self.last_reward = 0
        
        # For test-case inspections (compare both players' scores).
        self.player1_score = self.player2_score = 0

    # reset the env
    # return the initial observation
    # the player1 always take action first
    def reset(self):
        self.player1_init_pos, self.player2_init_pos = self.generate_init_pos()
        
        self.position = {
                self.player1: self.player1_init_pos,
                self.player2: self.player2_init_pos
        }
        
        self.health = {
            self.player1: LOREnv2.max_health,
            self.player2: LOREnv2.max_health
        }
        
        self.turn_reward_player1 = 0
        self.turn_reward_player2 = 0
        self.last_reward = 0
        
        return {
            self.player1: tuple(
                [
                    np.array([self.position[self.player1][0], self.position[self.player1][1]]),
                    np.array([self.position[self.player2][0], self.position[self.player2][1]]),
                    np.array([self.health[self.player1], self.health[self.player2]])
                ]
            )
        }
    
    
    def move_agent(self, player, opponent, action):
        if self.health[player] <= 0:  # no health no action
            return 0
        
        if action == LOREnv2.MOVEUP or action == LOREnv2.MOVEDOWN:
            new_x = self.position[player][0] + (1 if action == LOREnv2.MOVEDOWN else -1)
            if new_x < 0 or new_x >= LOREnv2.space_size_n: # invalid move
                return self.hit_wall_penalty
            elif (self.position[opponent][0] == new_x and self.position[opponent][1] == self.position[player][1]):
                return self.hit_wall_penalty
            else:
                self.position[player][0] = new_x
                return 0
                
        if action == LOREnv2.MOVELEFT or action == LOREnv2.MOVERIGHT:
            new_y = self.position[player][1] + (1 if action == LOREnv2.MOVERIGHT else -1)
            if new_y < 0 or new_y >= LOREnv2.space_size_n: # invalid move
                return self.hit_wall_penalty
            elif (self.position[opponent][1] == new_y and self.position[opponent][0] == self.position[player][0]):
                return self.hit_wall_penalty
            else:
                self.position[player][1] = new_y
                return 0
        
        return 0
    
    def is_adjacent(self):
        return (self.position[self.player1][0] == self.position[self.player2][0] and abs(self.position[self.player1][1] - self.position[self.player2][1]) <= 1) \
            or (self.position[self.player1][1] == self.position[self.player2][1] and abs(self.position[self.player1][0] - self.position[self.player2][0]) <= 1)
            

    #def attack_agent(self, action1, action2):
    #    gain1 = 0
    #    gain2 = 0
    #    
    #    if action1 == LOREnv2.ATTACK:
    #        defend1 = 0 if action2 == LOREnv2.DEFEND and random.random() < LOREnv2.defend_rate else 1
    #        hit1 =  0 if not(self.is_adjacent()) or random.random() < LOREnv2.attack_miss_rate else 1
    #        
    #        self.health[self.player2] = self.health[self.player2] - hit1 * defend1 * LOREnv2.attak_power
    #        gain1 = hit1 * defend1 * LOREnv2.attak_power
    #            
    #        
    #    if action2 == LOREnv2.ATTACK:
    #        defend2 = 0 if action1 == LOREnv2.DEFEND and random.random() < LOREnv2.defend_rate else 1
    #        hit2 =  0 if not(self.is_adjacent()) or random.random() < LOREnv2.attack_miss_rate else 1
    #                    
    #        self.health[self.player1] = self.health[self.player1] - hit2 * defend2 * LOREnv2.attak_power
    #        gain2 = hit2 * defend2 * LOREnv2.attak_power
    #    
    #    return gain1, gain2 
            
        
    
    def take_action(self, player, opponent, action):
        reward = 0
        
        if action == LOREnv2.ATTACK:
            hit1 =  0 if not(self.is_adjacent()) or random.random() < LOREnv2.attack_miss_rate else 1
            
            self.health[opponent] = self.health[opponent] - hit1  * LOREnv2.attak_power
            reward = hit1 * LOREnv2.attak_power
        else: # move
            reward = self.move_agent(player, opponent, action)
    
    
        # check health
        if self.health[player] <=0 and self.health[opponent] > 0:
            reward = -1 * LOREnv2.game_award
        
        if self.health[player] > 0 and self.health[opponent] <= 0:
            reward = LOREnv2.game_award
        
        if self.health[player] == 0 and self.health[opponent] == 0:
            reward = 0
    
        return reward
    
    
    # update state and observation based on the 2 actions
    def step(self, action_dict):
        # only one action each turn
        assert len(action_dict) == 1, action_dict
                
        if self.player1 in action_dict:
            player = self.player1
            opponent = self.player2
        else:
            player = self.player2
            opponent = self.player1

        reward = self.take_action(player, opponent, action_dict[player])
           
            
        # get the new obs
        obs = {
            opponent: tuple(
                [
                    np.array([self.position[opponent][0], self.position[opponent][1]]),
                    np.array([self.position[player][0], self.position[player][1]]),
                    np.array([self.health[opponent], self.health[player]])
                ]
            )
        }
        
        # get the reward
        rew = {
            opponent: -1 * reward + self.last_reward,
        }
        
        self.last_reward = reward
        
        done = {
            "__all__": self.health[self.player1] == 0 or self.health[self.player2] == 0
        }
        
        # it is required that when done["__all__"] == True, the obv/rew should include all live agent
        if done["__all__"]:
            obs[player] = tuple(
                [
                    np.array([self.position[player][0], self.position[player][1]]),
                    np.array([self.position[opponent][0], self.position[opponent][1]]),
                    np.array([self.health[player], self.health[opponent]])
                ]
            )
            
            rew[player] = reward
        

        if self.health[self.player2] == 0:
            self.player1_score += 1
        elif self.health[self.player1] == 0:
            self.player2_score += 1

        return obs, rew, done, {}
    


In [6]:
import random
from ray.rllib.policy.policy import Policy

class LORHeuristicReckLessOrCautious(Policy):
    """
    Heuristic policy
    Random pick between one of the following.
    
    cautious
    if self.health >= opponent.health and self.health > 1:
        if self and opponent is adjacent:
            attack
        else:
            move torwards the opponent
    else:
        if self and opponent is adjacent:
            move away from the opponent 
        else:
            attack
            
    reckless
    if self and opponent is adjacent:
        attack
    else
        move torwards the opponent
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.exploration = self._create_exploration()
    
    def take_cautious_action(self, obv):
        # each ob is np array (self.x, self.y, oponent.x, oppoennt.y, self.health, opponent.health)
        self_x = obv[0]
        self_y = obv[1]
        op_x = obv[2]
        op_y = obv[3]
        self_h = obv[4]
        op_h = obv[4]
        
        if self_h >= op_h and self_h > 1:
            if (self_x == op_x and abs(self_y - op_y) <= 1) or (self_y == op_y and abs(self_x - op_x) <= 1):
                return LOREnv2.ATTACK
            else:
                if self_x != op_x:
                    return LOREnv2.MOVEUP if self_x > op_x else LOREnv2.MOVEDOWN
                else:
                    return LOREnv2.MOVELEFT if self_y > op_y else LOREnv2.MOVERIGHT
        else:
            if (self_x == op_x and abs(self_y - op_y) <= 1) or (self_y == op_y and abs(self_x - op_x) <= 1):
                if self_x == op_x:
                    return LOREnv2.MOVEUP if self_x == LOREnv2.space_size_n -1  else LOREnv2.MOVEDOWN
                else:
                    return LOREnv2.MOVELEFT if self_y == LOREnv2.space_size_n -1  else LOREnv2.MOVERIGHT
            else:
                return LOREnv2.ATTACK

    
    def take_reckless_action(self, obv):
        # each ob is np array (self.x, self.y, oponent.x, oppoennt.y, self.health, opponent.health)
        self_x = obv[0]
        self_y = obv[1]
        op_x = obv[2]
        op_y = obv[3]
        self_h = obv[4]
        op_h = obv[4]
        
        move_x_first = 1 if random.random() < 0.5 else 0
        
        if (self_x == op_x and abs(self_y - op_y) <= 1) or (self_y == op_y and abs(self_x - op_x) <= 1):
            return LOREnv2.ATTACK
        else:
            if move_x_first:             
                if self_x != op_x:
                    return LOREnv2.MOVEUP if self_x > op_x else LOREnv2.MOVEDOWN
                else:
                    return LOREnv2.MOVELEFT if self_y > op_y else LOREnv2.MOVERIGHT
            else:
                if self_y != op_y:
                    return LOREnv2.MOVELEFT if self_y > op_y else LOREnv2.MOVERIGHT
                else:
                    return LOREnv2.MOVEUP if self_x > op_x else LOREnv2.MOVEDOWN
    

    def compute_actions(self,
                        obs_batch,
                        state_batches=None,
                        prev_action_batch=None,
                        prev_reward_batch=None,
                        info_batch=None,
                        episodes=None,
                        **kwargs):

        return [self.take_reckless_action(x) if random.random() < 0.5 else self.take_cautious_action(x)  for x in obs_batch], [], {}
    
    def learn_on_batch(self, samples):
        pass

    def get_weights(self):
        pass

    def set_weights(self, weights):
        pass


In [7]:
import ray
from ray.rllib.agents.dqn import DQNTrainer

ray.shutdown()
ray.init()

def select_policy(agent_id):
    if agent_id == "player1":
        return "learned"
    else:
        return "LORHeuristicReckLessOrCautious"

env = LOREnv2({})
    
config = {
    "env": LOREnv2,
    "gamma": 0.9,
    "num_workers": 0,
    "num_envs_per_worker": 4,
    "rollout_fragment_length": 10,
    "train_batch_size": 500,
    "multiagent": {
        "policies_to_train": ["learned"],
        "policies": {
            "LORHeuristicReckLessOrCautious": (LORHeuristicReckLessOrCautious, env.observation_space, env.action_space, {}),
            "learned": (None, env.observation_space, env.action_space, {
                "model": {
                        "use_lstm": True
                },
            }),
        },
        "policy_mapping_fn": select_policy,
    },
}

trainer_obj2 = DQNTrainer(config=config)
env = trainer_obj2.workers.local_worker().env
for _ in range(50):
    results = trainer_obj2.train()
    #print(results)
    
    #if _ % 100 == 0:
    print(env.player1_score, env.player2_score)

2020-06-23 03:59:18,588	INFO resource_spec.py:212 -- Starting Ray with 22.75 GiB memory available for workers and up to 11.4 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-23 03:59:18,888	INFO services.py:1170 -- View the Ray dashboard at localhost:8271
2020-06-23 03:59:21,154	WARNING trainer_template.py:124 -- The experimental distributed execution API is enabled for this algorithm. Disable this by setting 'use_exec_api': False.
2020-06-23 03:59:21,156	INFO trainable.py:217 -- Getting current IP.
2020-06-23 03:59:21,157	WARNING util.py:37 -- Install gputil for GPU system monitoring.


0 16
2 35
5 50
6 65
11 79
16 94
26 101
36 109
49 115
66 120
87 123
105 124
125 129
136 132
148 140
167 143
182 146
191 148
192 151
209 153
224 153
230 155
230 155
236 157
240 158
243 160
246 161
254 165
272 166
277 166
282 167
293 168
295 169
310 169
316 171
318 173
322 174
325 175
329 177
338 178
339 181
343 183
343 183
350 183
352 183
354 184
357 185
364 186
377 187
383 188


In [8]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
def print_obv(env, obv, size):
    for i in range(size):
        for j in range(size):
            if i == env.position[env.player1][0] and j == env.position[env.player1][1]:
                print(f"{bcolors.FAIL}{env.health[env.player1]}{bcolors.ENDC}", end = "")
            elif i == env.position[env.player2][0] and j == env.position[env.player2][1]:
                print(f"{bcolors.OKBLUE}{env.health[env.player2]}{bcolors.ENDC}", end = "")
            else:
                print("-", end = "")
        print("")

        
    
def simulateTurns(env, trainer1, size, base_policy, max_turn = 100):
    obv = env.reset()
    done = {"__all__" : False}
    turn = 0
    
    while done["__all__"] == False and turn < max_turn:
        # print ovb
        print_obv(env, obv, size)
        
        # player1 take action
        a1 = trainer1.compute_action(observation = obv[env.player1], policy_id = "learned")
        
        print("".join(['*']*(size)) , end = " action ")
        print(f"{bcolors.FAIL}{env.action_string[a1]}{bcolors.ENDC}", end = " , ")
        print(f"{bcolors.OKBLUE}WAIT{bcolors.ENDC}")
        
        # update obv
        obv, reward, done, info = env.step({env.player1: a1})
        
                
        if done["__all__"] == False:
            # print new obv
            print_obv(env, obv, size)

            # player2 take action
            a2 = trainer1.compute_action(observation = obv[env.player2], policy_id = base_policy)

            print("".join(['*']*(size)) , end = " action ")
            print(f"{bcolors.FAIL}WAIT{bcolors.ENDC}", end = " , ")
            print(f"{bcolors.OKBLUE}{env.action_string[a2]}{bcolors.ENDC}")

            # update obv
            obv, reward, done, info = env.step({env.player2: a2})
                
        turn += 1
    
    print_obv(env, obv, size)

the stratgy of the player1 is easy, attack as long as its health is more or equal.

In [10]:
trainer_obj2.get_policy("learned").config['explore'] = False
sim_env = LOREnv2({})

simulateTurns(sim_env, trainer_obj2, 3, "LORHeuristicReckLessOrCautious")

---
-33
---
*** action Attack , WAIT
---
-23
---
*** action WAIT , Attack
---
-22
---
*** action Attack , WAIT
---
-22
---
*** action WAIT , Attack
---
-21
---
*** action MoveDown , WAIT
---
-2-
--1
*** action WAIT , MoveDown
---
---
-21
*** action MoveUp , WAIT
---
--1
-2-
*** action WAIT , MoveUp
---
-21
---
*** action MoveDown , WAIT
---
-2-
--1
*** action WAIT , MoveDown
---
---
-21
*** action MoveUp , WAIT
---
--1
-2-
*** action WAIT , MoveRight
---
--1
--2
*** action MoveLeft , WAIT
---
-1-
--2
*** action WAIT , MoveLeft
---
-1-
-2-
*** action Attack , WAIT
---
-1-
-1-
*** action WAIT , Attack
---
-0-
-1-


in this round of game, player1 missed one attack so that the health gest to 1:2. the player1 starts to run away and keeps run away...

In [11]:
simulateTurns(sim_env, trainer_obj2, 3, "LORHeuristicReckLessOrCautious")

--3
-3-
---
*** action Attack , WAIT
--3
-3-
---
*** action WAIT , MoveUp
-33
---
---
*** action Attack , WAIT
-23
---
---
*** action WAIT , Attack
-22
---
---
*** action Attack , WAIT
-22
---
---
*** action WAIT , Attack
-21
---
---
*** action MoveDown , WAIT
-2-
--1
---
*** action WAIT , MoveDown
---
-21
---
*** action MoveDown , WAIT
---
-2-
--1
*** action WAIT , MoveDown
---
---
-21
*** action MoveUp , WAIT
---
--1
-2-
*** action WAIT , MoveUp
---
-21
---
*** action MoveDown , WAIT
---
-2-
--1
*** action WAIT , MoveDown
---
---
-21
*** action MoveUp , WAIT
---
--1
-2-
*** action WAIT , MoveUp
---
-21
---
*** action MoveDown , WAIT
---
-2-
--1
*** action WAIT , MoveDown
---
---
-21
*** action MoveUp , WAIT
---
--1
-2-
*** action WAIT , MoveUp
---
-21
---
*** action MoveDown , WAIT
---
-2-
--1
*** action WAIT , MoveRight
---
--2
--1
*** action MoveLeft , WAIT
---
--2
-1-
*** action WAIT , MoveDown
---
---
-12
*** action MoveUp , WAIT
---
-1-
--2
*** action WAIT , MoveUp
---
-12
---
*

In [14]:
simulateTurns(sim_env, trainer_obj2, 3, "LORHeuristicReckLessOrCautious")

---
-3-
-3-
*** action Attack , WAIT
---
-3-
-3-
*** action WAIT , Attack
---
-3-
-3-
*** action Attack , WAIT
---
-3-
-2-
*** action WAIT , Attack
---
-2-
-2-
*** action Attack , WAIT
---
-2-
-2-
*** action WAIT , Attack
---
-1-
-2-
*** action Attack , WAIT
---
-1-
-1-
*** action WAIT , MoveRight
---
-1-
--1
*** action Attack , WAIT
---
-1-
--1
*** action WAIT , MoveUp
---
-11
---
*** action Attack , WAIT
---
-10
---
